# MozzieDrome plotting
### Linhan Dong, Duvall Lab

### Behavior classification heatmap

In [ ]:
import h5py
import numpy as np
from scipy.signal import savgol_filter
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
behavior_input = '/Users/donglinhan/Desktop/SLEAP/WT_DD2_all_behavior_modified_again.xlsx'
behaviors = pd.read_excel(behavior_input)

# Create a colormap based on the defined colors
colors = ['white', '#247de3', '#e32424']
cmap = plt.cm.colors.ListedColormap(colors)

#pdf_pages = PdfPages('Trial-PDF_behavior_plot.pdf')
plot_width = 15
plot_height = 0.25
total = len(behaviors.columns)
fig, axes = plt.subplots(nrows=total, ncols=1, figsize=(plot_width, total*plot_height))
for i, column_name in enumerate(behaviors.columns):
    ax = axes[i]
    behavior_array = np.array(behaviors[column_name]) 
    ax.imshow(behavior_array[:,np.newaxis].T, aspect='auto', cmap= cmap, vmin=0, vmax=2)
    ax.axis("off")
    ax.axvline(x=22.5, color='black', linestyle='-', linewidth = 0.25)
    #ax.set_title(column_name, loc='left')
    print(i)
plt.savefig('classification_WT_DD2.svg', bbox_inches='tight')
plt.close()

In [ ]:
#SORT COLUMNS
import h5py
import numpy as np
from scipy.signal import savgol_filter
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
behavior_input = '/Users/donglinhan/Desktop/SLEAP/PDF_DD2_all_behavior_modified_again.xlsx'
raw_behaviors = pd.read_excel(behavior_input)

regions = [
    (0, 29),   
    (29, 58),  
    (58, 87), 
    (87, 116),
    (116, 145),
    (145, 174),
    (174, 203),
    (203, 232),
]

sorted_df = pd.DataFrame()

for start, end in regions:
    region_columns = raw_behaviors.iloc[:, start:end]
    non_zero_counts = (region_columns != 0).sum().sort_values(ascending=False)
    sorted_region = region_columns[non_zero_counts.index]
    sorted_df = pd.concat([sorted_df, sorted_region], axis=1)

# Display the first few rows of the sorted dataframe
sorted_df.head()

behaviors = sorted_df

# Create a colormap based on the defined colors
colors = ['white', '#247de3', '#e32424']
cmap = plt.cm.colors.ListedColormap(colors)

#pdf_pages = PdfPages('Trial-PDF_behavior_plot.pdf')
plot_width = 15
plot_height = 0.25
total = len(behaviors.columns)
fig, axes = plt.subplots(nrows=total, ncols=1, figsize=(plot_width, total*plot_height))
for i, column_name in enumerate(behaviors.columns):
    ax = axes[i]
    behavior_array = np.array(behaviors[column_name]) 
    ax.imshow(behavior_array[:,np.newaxis].T, aspect='auto', cmap= cmap, vmin=0, vmax=2)
    ax.axis("off")
    ax.axvline(x=22.5, color='black', linestyle='-', linewidth = 0.25)
    #ax.set_title(column_name, loc='left')
    print(i)
plt.savefig('sorted_classification_Hom_DD.svg', bbox_inches='tight')
plt.close()

### Representative traces

In [ ]:
import h5py
import numpy as np
from scipy.interpolate import interp1d

filename_array = ["/Users/donglinhan/Desktop/SLEAP/FinalH5/WT-LD-1/WT-ZT0-LD2-0817_01.842.h5", "/Users/donglinhan/Desktop/SLEAP/FinalH5/WT-LD-1/WT-ZT6-LD2-0829_01.849.h5", "/Users/donglinhan/Desktop/SLEAP/FinalH5/WT-LD-1/WT-ZT12-LD1-0829_01.958.h5", "/Users/donglinhan/Desktop/SLEAP/FinalH5/WT-LD-1/WT-ZT18-LD2-0829_01.830.h5"]
delay_array = [1.842, 1.849, 1.958, 1.830]
output_array = ["ZT0_new_representative_plots_solidcolors.png", "ZT6_new_representative_plots_solidcolors.png", "ZT12_new_representative_plots_solidcolors.png", "ZT18_new_representative_plots_solidcolors.png"]

for a in range(0,4):
    filename = filename_array[a]
    delay = delay_array[a]
    output_name = output_array[a]
    with h5py.File(filename, "r") as f:
        dset_names = list(f.keys())
        locations = f["tracks"][:].T
        node_names = [n.decode() for n in f["node_names"][:]] 
    def fill_missing(Y, kind="linear"):
        initial_shape = Y.shape
        Y = Y.reshape((initial_shape[0], -1))
        for i in range(Y.shape[-1]):
            y = Y[:, i]
            x = np.flatnonzero(~np.isnan(y))
            f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)
            xq = np.flatnonzero(np.isnan(y))
            y[xq] = f(xq)
            mask = np.isnan(y)
            y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])
            Y[:, i] = y
        Y = Y.reshape(initial_shape)
        return Y
    locations = fill_missing(locations)
    print (locations.shape)
    
    BODY_INDEX = 0
    
    body_loc = locations[:, BODY_INDEX, :, :]
    
    import seaborn as sns
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_pdf import PdfPages
    import pandas as pd
    from scipy.signal import savgol_filter
    import seaborn as sns
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_pdf import PdfPages
    def smooth_diff(node_loc, win=25, poly=3):
        node_loc_vel = np.zeros_like(node_loc)
        for c in range(node_loc.shape[-1]):
            node_loc_vel[:, c] = savgol_filter(node_loc[:, c], win, poly, deriv=1)
        node_vel = np.linalg.norm(node_loc_vel,axis=1)
        return node_vel
    thx_vel_fly = pd.DataFrame()
    for i in range (locations.shape[3]):
        thx_vel_fly[i] = smooth_diff(body_loc[:, :, i])
    
    normalized = 5 - thx_vel_fly
    normalized [normalized < 1] = 1
    plot_width = 2.8
    plot_height = 2

    colors = ['#915ebb', '#3d5090', '#b34a9c', '#a4a4a4', '#4d91df', '#5ea965', '#e6d966', '#e76767', '#c97295', '#85a443']
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(3*plot_width, 1*plot_height))
    
    delay_frame = int(delay * 60)
    co2_frame = 7200 - delay_frame
    
    for ax in axes:
        # Remove axis labels and ticks
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        #ax.set_facecolor('#f0f0f0')
        
    for spine in ax.spines.values():
        spine.set_linewidth(1)
    
    for ax in axes:
        ax.set_xlim(0, 1120)
        ax.set_ylim(0, 800)
    
    ax = axes[0]
    for i in range(locations.shape[3]):
        for j in range(co2_frame):
            ax.plot([body_loc[j, 0, i], body_loc[j+1, 0, i]], [body_loc[j, 1, i], body_loc[j+1, 1, i]], color=colors[i], alpha=1, linewidth=normalized[i][j])
    
    ax = axes[1]
    for i in range(locations.shape[3]):
        for j in range(co2_frame, co2_frame+7200):
            ax.plot([body_loc[j, 0, i], body_loc[j+1, 0, i]], [body_loc[j, 1, i], body_loc[j+1, 1, i]], color=colors[i], alpha=1, linewidth=normalized[i][j])
    
    ax = axes[2]
    for i in range(locations.shape[3]):
        for j in range(co2_frame + 28800, co2_frame+36000):
            ax.plot([body_loc[j, 0, i], body_loc[j+1, 0, i]], [body_loc[j, 1, i], body_loc[j+1, 1, i]], color=colors[i], alpha=1, linewidth=normalized[i][j])
    
    plt.tight_layout()
    
    fig.savefig(output_name, transparent=True, dpi=600)
    
    plt.close()

### Behavior classification examples

In [3]:
import h5py
import numpy as np
from scipy.interpolate import interp1d
import seaborn as sns
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

filename = "/Users/donglinhan/Desktop/SLEAP/FinalH5/WT-LD-1/WT-ZT0-LD2-0907_01.726.h5"
delay = 1.726
with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

def fill_missing(Y, kind="linear"):
    initial_shape = Y.shape
    Y = Y.reshape((initial_shape[0], -1))
    for i in range(Y.shape[-1]):
        y = Y[:, i]
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])
        Y[:, i] = y
    Y = Y.reshape(initial_shape)
    return Y

locations = fill_missing(locations)
print (locations.shape)
BODY_INDEX = 0
body_loc = locations[:, BODY_INDEX, :, :]

def smooth_diff(node_loc, win=25, poly=3):

    node_loc_vel = np.zeros_like(node_loc)
    for c in range(node_loc.shape[-1]):
        node_loc_vel[:, c] = savgol_filter(node_loc[:, c], win, poly, deriv=1)
    node_vel = np.linalg.norm(node_loc_vel,axis=1)
    return node_vel

thx_vel_fly = pd.DataFrame()
for i in range (locations.shape[3]):
    thx_vel_fly[i] = smooth_diff(body_loc[:, :, i])

plot_width = 2.8
plot_height = 2

pdf = PdfPages('example_scatter_plots.pdf')

fig, axes = plt.subplots(nrows=1, ncols=10, figsize=(10*plot_width, 1*plot_height))

delay_frame = int(delay * 60)
co2_frame = 7200 - delay_frame

for ax in axes:
    # Remove axis labels and ticks
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    #ax.set_facecolor('#f0f0f0')
    
for spine in ax.spines.values():
    spine.set_linewidth(1)

for ax in axes:
    ax.set_xlim(0, 1120)
    ax.set_ylim(0, 800)

delay = 1.726
frame_count = locations.shape[0]
delay_frame = int(delay * 60)
window_size = 300
start_frame = 300 - delay_frame
vector_windows = (frame_count + delay_frame - 241) // window_size
i = 4

for k in range(94, 100):
    frame_start = start_frame + k * window_size  # Previous coordinates
    frame_end = start_frame + (k+1) * window_size
    ax = axes[k - 94]
    for j in range(frame_start, frame_end):
        ax.scatter(body_loc[j, 0, i], body_loc[j, 1, i], s=20, edgecolor='gray',linewidth=0.25, facecolor='none', alpha=0.25)


plt.tight_layout()
pdf.savefig(fig)
plt.close(fig)
pdf.close()

(43215, 1, 2, 10)


In [4]:
import h5py
import numpy as np
from scipy.interpolate import interp1d
import seaborn as sns
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

filename = "/Users/donglinhan/Desktop/SLEAP/FinalH5/WT-LD-1/WT-ZT0-LD2-0907_01.726.h5"
delay = 1.726
with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

def fill_missing(Y, kind="linear"):
    initial_shape = Y.shape
    Y = Y.reshape((initial_shape[0], -1))
    for i in range(Y.shape[-1]):
        y = Y[:, i]
        x = np.flatnonzero(~np.isnan(y))
        f = interp1d(x, y[x], kind=kind, fill_value=np.nan, bounds_error=False)
        xq = np.flatnonzero(np.isnan(y))
        y[xq] = f(xq)
        mask = np.isnan(y)
        y[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), y[~mask])
        Y[:, i] = y
    Y = Y.reshape(initial_shape)
    return Y

locations = fill_missing(locations)
print (locations.shape)
BODY_INDEX = 0
body_loc = locations[:, BODY_INDEX, :, :]

def smooth_diff(node_loc, win=25, poly=3):

    node_loc_vel = np.zeros_like(node_loc)
    for c in range(node_loc.shape[-1]):
        node_loc_vel[:, c] = savgol_filter(node_loc[:, c], win, poly, deriv=1)
    node_vel = np.linalg.norm(node_loc_vel,axis=1)
    return node_vel

thx_vel_fly = pd.DataFrame()
for i in range (locations.shape[3]):
    thx_vel_fly[i] = smooth_diff(body_loc[:, :, i])
plot_width = 2.8
plot_height = 2

pdf = PdfPages('example_line_plots.pdf')

colors = ['#6baed6', '#74c476', 'gray', 'black', 'black', '#ffff99', '#a1d99b', 'black', '#b2df8a', '#31a354']
fig, axes = plt.subplots(nrows=1, ncols=10, figsize=(10*plot_width, 1*plot_height))

delay_frame = int(delay * 60)
co2_frame = 7200 - delay_frame


for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
  
for spine in ax.spines.values():
    spine.set_linewidth(1)

for ax in axes:
    ax.set_xlim(0, 1120)
    ax.set_ylim(0, 800)

delay = 1.726
frame_count = locations.shape[0]
delay_frame = int(delay * 60)
window_size = 300
start_frame = 300 - delay_frame
vector_windows = (frame_count + delay_frame - 241) // window_size
for k in range(94, 104):
    frame_start = start_frame + k * window_size  # Previous coordinates
    frame_end = start_frame + (k+1) * window_size
    ax = axes[k - 94]
    i = 4
    for j in range(frame_start, frame_end):
        ax.plot([body_loc[j, 0, i], body_loc[j+1, 0, i]], [body_loc[j, 1, i], body_loc[j+1, 1, i]], color=colors[i], alpha=1, linewidth=0.5)

plt.tight_layout()
pdf.savefig(fig)
plt.close(fig)
pdf.close()

(43215, 1, 2, 10)
